In [1]:
import os
from typing import Tuple

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from IPython.display import display

import openai


In [2]:
with open('secrets.txt', 'r') as f:
    openai.api_key = f.read().split('=')[1].strip()


In [3]:
base_instruction = "You are a professional legal translator from Greek into German. You don't produce regular translations, though. You produce"

instructions = {
    'τεκμηριωτική':
        f"{base_instruction} documentary translations. Documentary translation aims at producing in the target language a kind of document of (certain aspects of) a communicative interaction in which a source-culture sender communicates with a source-culture audience via the source text under source-culture conditions.",
    'ομοιολειτουργική':
        f"{base_instruction} equifunctional translations. If the target-text function is the same as that of the source text, we can speak of an equifunctional translation. Equifunctional translations are found in the area of technical texts, computer manuals, and other pragmatic texts such as instructions for use, recipes, tourist information texts, and information on products."
}


In [4]:
examples = pd.read_csv('examples.csv', encoding='utf-8')
examples = examples[examples['version'] == 'ελλΑΚ-1940']
display(examples.dtypes)
examples


id           int64
lang        object
version     object
sentence    object
dtype: object

,id,lang,version,sentence
0,1,EL,ελλΑΚ-1940,Κατὰ τὴν ἑρμηνείαν τῆς δηλώσεως βουλήσεως ἀναζ...
4,2,EL,ελλΑΚ-1940,Διὰ τὰς προθεσμίας τὰς καθοριζομένας διὰ νόμου...
8,3,EL,ελλΑΚ-1940,Ἡ μετὰ τὴν ἐπιχείρησιν τῆς δικαιπραξὶας παρεχο...
12,4,EL,ελλΑΚ-1940,Ἅμα τῷ θανάτῳ τοῦ προσώπου ἡ περιουσία αὐτοῦ ὡ...
16,5,EL,ελλΑΚ-1940,Κληρονόμος δύναται να γείνῃ μόνον ὁ κατὰ τὸν χ...


In [5]:
def gpt4(
        prompt,
        *,
        model=None,
        system=None,
        max_tokens=100,
        temperature=0.5,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=None
) -> Tuple[str, dict]:
    """Generate chat completion using GPT-4."""

    parameters = {
        'model': model,
        'max_tokens': max_tokens,
        'temperature': temperature,
        'top_p': top_p,
        'frequency_penalty': frequency_penalty,
        'presence_penalty': presence_penalty,
    }

    completion = openai.ChatCompletion.create(
        messages = [{'role': 'user', 'content': prompt},
                    {'role': 'system', 'content': system}],
        stop = stop,
        **parameters
    )

    return completion.choices[0].message['content'], parameters


In [6]:
for translation_type, instruction in instructions.items():
    print(f"Generating completion for {translation_type}...")
    model = 'gpt-4-0613'
    for sentence in examples['sentence']:
        print(f"Sentence: {sentence!r}")
        completion, parameters = gpt4(
            prompt = sentence,
            model = model,
            system = instruction,
            stop = '\n\n'
        )
        print(f"Completion: {completion!r}", end='\n\n')
        examples.loc[examples['sentence'] == sentence, f'{translation_type}'] = completion
        examples.loc[examples['sentence'] == sentence, 'parameters'] = str(parameters)
    print("Done.\n")


Generating completion for τεκμηριωτική...
Sentence: 'Κατὰ τὴν ἑρμηνείαν τῆς δηλώσεως βουλήσεως ἀναζητεῖται ἤ ἀληθὴς βούλησις ἄνευ προσηλώσεως εἱς τὰς λέξεις. (ΑΚ 173)'
Completion: 'In der Interpretation der Willenserklärung wird entweder ein wahrer Wille ohne Fixierung auf die Worte gesucht. (AK 173)'

Sentence: 'Διὰ τὰς προθεσμίας τὰς καθοριζομένας διὰ νόμου, δικαστικῆς αποφάσεως ἤ δικαιοπραξίας ἰσχύουσιν αἱ ἑρμηνευτικαὶ διατάξεις τῶν ἄρθρων 241 ἔως 246. (ΑΚ 240)'
Completion: 'Durch die gesetzlich festgelegten Fristen gelten die Auslegungsbestimmungen der Artikel 241 bis 246 im Falle eines Gerichtsurteils oder einer rechtlichen Handlung. (AK 240)'

Sentence: 'Ἡ μετὰ τὴν ἐπιχείρησιν τῆς δικαιπραξὶας παρεχομένη συγκατάθεσις (ἔγκρισις), ἐφ’ ὄσον δὲν ὁρίζεται τὸ ἐναντὶον, ἀνατρέχει εἰς τον χρόνον τῆς δικαιοπραξίας. (ΑΚ 238)'
Completion: 'Die nach der Durchführung des Rechtsverfahrens gewährte Zustimmung (Genehmigung), soweit nichts Gegenteiliges festgelegt ist, bezieht sich auf die Zeit d

In [7]:
examples = pd.concat([examples, examples['parameters'].apply(eval).apply(pd.Series)], axis=1)
examples = examples.drop(columns=['parameters'])
examples.to_csv('examples_gpt4-translated.csv', encoding='utf-8', index=False)
examples


,id,lang,version,sentence,τεκμηριωτική,ομοιολειτουργική,model,max_tokens,temperature,top_p,frequency_penalty,presence_penalty
0,1,EL,ελλΑΚ-1940,Κατὰ τὴν ἑρμηνείαν τῆς δηλώσεως βουλήσεως ἀναζ...,In der Interpretation der Willenserklärung wir...,Bei der Interpretation der Willenserklärung wi...,gpt-4-0613,100,0.5,1.0,0.0,0.0
4,2,EL,ελλΑΚ-1940,Διὰ τὰς προθεσμίας τὰς καθοριζομένας διὰ νόμου...,Durch die gesetzlich festgelegten Fristen gelt...,"Für die durch Gesetz, Gerichtsentscheidung ode...",gpt-4-0613,100,0.5,1.0,0.0,0.0
8,3,EL,ελλΑΚ-1940,Ἡ μετὰ τὴν ἐπιχείρησιν τῆς δικαιπραξὶας παρεχο...,Die nach der Durchführung des Rechtsverfahrens...,"Die Zustimmung (Genehmigung), die nach der Dur...",gpt-4-0613,100,0.5,1.0,0.0,0.0
12,4,EL,ελλΑΚ-1940,Ἅμα τῷ θανάτῳ τοῦ προσώπου ἡ περιουσία αὐτοῦ ὡ...,Mit dem Tod einer Person geht sein gesamtes Ve...,Mit dem Tod einer Person geht ihr gesamtes Ver...,gpt-4-0613,100,0.5,1.0,0.0,0.0
16,5,EL,ελλΑΚ-1940,Κληρονόμος δύναται να γείνῃ μόνον ὁ κατὰ τὸν χ...,"Ein Erbe kann nur jemand sein, der zur Zeit de...","Ein Erbe kann nur derjenige werden, der zum Ze...",gpt-4-0613,100,0.5,1.0,0.0,0.0


In [10]:
with pd.option_context('display.max_colwidth', None):
    display(examples[['id', 'sentence', 'τεκμηριωτική', 'ομοιολειτουργική']])


,id,sentence,τεκμηριωτική,ομοιολειτουργική
0,1,Κατὰ τὴν ἑρμηνείαν τῆς δηλώσεως βουλήσεως ἀναζητεῖται ἤ ἀληθὴς βούλησις ἄνευ προσηλώσεως εἱς τὰς λέξεις. (ΑΚ 173),In der Interpretation der Willenserklärung wird entweder ein wahrer Wille ohne Fixierung auf die Worte gesucht. (AK 173),Bei der Interpretation der Willenserklärung wird der wahre Wille ohne Fixierung auf die Worte gesucht. (AK 173)
4,2,"Διὰ τὰς προθεσμίας τὰς καθοριζομένας διὰ νόμου, δικαστικῆς αποφάσεως ἤ δικαιοπραξίας ἰσχύουσιν αἱ ἑρμηνευτικαὶ διατάξεις τῶν ἄρθρων 241 ἔως 246. (ΑΚ 240)",Durch die gesetzlich festgelegten Fristen gelten die Auslegungsbestimmungen der Artikel 241 bis 246 im Falle eines Gerichtsurteils oder einer rechtlichen Handlung. (AK 240),"Für die durch Gesetz, Gerichtsentscheidung oder Rechtshandlung festgelegten Fristen gelten die Auslegungsbestimmungen der Artikel 241 bis 246. (AK 240)"
8,3,"Ἡ μετὰ τὴν ἐπιχείρησιν τῆς δικαιπραξὶας παρεχομένη συγκατάθεσις (ἔγκρισις), ἐφ’ ὄσον δὲν ὁρίζεται τὸ ἐναντὶον, ἀνατρέχει εἰς τον χρόνον τῆς δικαιοπραξίας. (ΑΚ 238)","Die nach der Durchführung des Rechtsverfahrens gewährte Zustimmung (Genehmigung), soweit nichts Gegenteiliges festgelegt ist, bezieht sich auf die Zeit des Rechtsverfahrens zurück. (AK 238)","Die Zustimmung (Genehmigung), die nach der Durchführung der Rechtshandlung gewährt wird, geht, sofern nicht anders angegeben, auf den Zeitpunkt der Rechtshandlung zurück. (AK 238)"
12,4,Ἅμα τῷ θανάτῳ τοῦ προσώπου ἡ περιουσία αὐτοῦ ὡς σύνολον (κληρονομία) μεταβαίνει ἐκ τοῦ νόμου ἤ ἐκ διαθήκης εἰς ἕν ἤ πλείονα πρόσωπα (κληρονόμοι). (ΑΚ 1710),Mit dem Tod einer Person geht sein gesamtes Vermögen (Erbschaft) nach dem Gesetz oder durch ein Testament auf eine oder mehrere Personen (Erben) über. (AK 1710),Mit dem Tod einer Person geht ihr gesamtes Vermögen (Erbschaft) nach dem Gesetz oder durch ein Testament auf eine oder mehrere Personen (Erben) über. (AK 1710)
16,5,Κληρονόμος δύναται να γείνῃ μόνον ὁ κατὰ τὸν χρόνον τῆς ἐπαγωγῆς τῆς κληρονομίας ἐν ζωῇ εὑρισκόμενος ἤ τουλάχιστον συνειλημμένος. (ΑΚ 1711),"Ein Erbe kann nur jemand sein, der zur Zeit der Erbschaftsannahme lebt oder zumindest empfangen wurde. (ABGB 1711)","Ein Erbe kann nur derjenige werden, der zum Zeitpunkt der Erbschaftsankündigung lebt oder zumindest gezeugt wurde. (AK 1711)"


In [ ]:
# create a class similar to sklearn's GridSearchCV to run the model with different parameters

class GPT4GridSearchCV:
    def __init__(self,
                 model=None,
                 system=None,
                 max_tokens=None,
                 temperature=None,
                 top_p=None,
                 frequency_penalty=None,
                 presence_penalty=None,
                 stop=None
        ):
        self.model = model
        self.system = system
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.top_p = top_p
        self.frequency_penalty = frequency_penalty
        self.presence_penalty = presence_penalty
        self.stop = stop

    def fit(self, X, y=None):
        self.results_ = []
        for i, sentence in enumerate(X, start=1):
            maxlen = str(len(X))
            print(f"Sentence {i:>{len(maxlen)}}/{len(X)}: {sentence!r}")
            completion, parameters = gpt4(
                prompt = sentence,
                model = self.model,
                system = self.system,
                max_tokens = self.max_tokens,
                temperature = self.temperature,
                top_p = self.top_p,
                frequency_penalty = self.frequency_penalty,
                presence_penalty = self.presence_penalty,
                stop = self.stop
            )
            print(f"Completion: {completion}", end='\n\n')
            self.results_.append((sentence, completion, parameters))
        return self

    def predict(self, X):
        return [result[1] for result in self.results_]

    def score(self, X, y=None):
        return 0

    def get_params(self, deep=True):
        return {
            'model': self.model,
            'system': self.system,
            'max_tokens': self.max_tokens,
            'temperature': self.temperature,
            'top_p': self.top_p,
            'frequency_penalty': self.frequency_penalty,
            'presence_penalty': self.presence_penalty,
            'stop': self.stop,
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
